In [1]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
#dummy dataset
data = {
    "Sleep_Hours": [6,7,8,5,7,5,6,7,6,5,8,9,5,6,7],
    "Daily_Steps:": [5000,7000,8000,3500,4000,5000,6000,7000,4500,7000,4600,6000,5000,6000,8000],
    "Calories_Intake": [2000, 2200, 2100, 2500, 2300, 2000, 1900, 2600, 2250, 2150, 2050, 2000, 2300, 2450, 2100],
    "BMI": [24, 23, 22, 26, 23, 25, 21, 27, 23, 24, 22, 25, 23, 26, 23]
}

df = pd.DataFrame(data)
df

,Sleep_Hours,Daily_Steps:,Calories_Intake,BMI
0,6,5000,2000,24
1,7,7000,2200,23
2,8,8000,2100,22
3,5,3500,2500,26
4,7,4000,2300,23
5,5,5000,2000,25
6,6,6000,1900,21
7,7,7000,2600,27
8,6,4500,2250,23
9,5,7000,2150,24


In [6]:
#identify target column

X = df.drop("BMI", axis=1).values
Y = df["BMI"].values.reshape(-1,1)

In [7]:
#Apply standardscaler

scaler_x = StandardScaler()
scaler_y = StandardScaler()

In [8]:
X_scaled = scaler_x.fit_transform(X)
Y_scaled = scaler_y.fit_transform(Y)

In [10]:
#convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
Y_tensor = torch.tensor(Y_scaled, dtype=torch.float32)

In [14]:
#Simple linear regression model
model = nn.Linear(3,1)

#loss and optimizer

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

#training

epochs = 500
for epoch in range(epochs):
    y_pred = model(X_tensor)
    loss = criterion(y_pred, Y_tensor)

    #baclward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(epoch+1) % 100 ==0:
        print(f"Eposch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

    

Eposch 100/500, Loss: 0.5899
Eposch 200/500, Loss: 0.5864
Eposch 300/500, Loss: 0.5863
Eposch 400/500, Loss: 0.5863
Eposch 500/500, Loss: 0.5863


In [22]:
#predictions

with torch.no_grad():
    y_pred_scaled = model(X_tensor)
    y_pred_original = scaler_y.inverse_transform(y_pred_scaled.numpy())


mse = mean_squared_error(Y, y_pred_original)
r2 = r2_score(Y, y_pred_original)

print(f"mse:, {mse:.4f}")
print(f"r2_score: {r2:.4f}")

mse:, 1.5792
r2_score: 0.4137


In [24]:
#predicitions  for new sample

new_sample = torch.tensor([[7,7000,2200]], dtype = torch.float32)

new_sample_sc = torch.tensor(scaler_x.transform(new_sample), dtype=torch.float32)

#predictions

with torch.no_grad():
    bmi_sc = model(new_sample_sc)
    bmi = scaler_y.inverse_transform(bmi_sc.numpy())


print("predicted:BMI", round(bmi[0][0],2))

predicted:BMI 23.81
